Получаем данные с coinmarketcap.com

In [ ]:
import requests
import json

c_url = "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit=1500&sortBy=market_cap&sortType=desc&convert=USD,BTC,ETH&cryptoType=all&tagType=all&audited=false&aux=ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap"
c_resp = requests.get(c_url)
c_resp.raise_for_status() # для проверки ошибок
c_data = c_resp.json()["data"]["cryptoCurrencyList"]

Получаем данные с simpleswap

In [7]:
s_url = "https://simpleswap.io/api/v3/currencies?fixed=false&includeDisabled=false"
s_resp = requests.get(s_url)
s_resp.raise_for_status()
s_data = s_resp.json()

Собираем список монет simpleswap для сравнения с coinmarketcap

In [6]:
s_symbols = {coin['symbol'].upper() for coin in s_data}

Находим монеты из coinmarketcap, которых нет в simpleswap (volume24h - берем в usd)

In [15]:
for i in c_data:
    print(i["quotes"])

[{'name': 'BTC', 'price': 1, 'volume24h': 1256159.5459573572, 'volume7d': 8793101.503173364, 'volumePercentChange': 52.3325, 'volume30d': 30231967.247920286, 'marketCap': 19950356.0, 'selfReportedMarketCap': 0.0, 'percentChange1h': 0, 'percentChange24h': 0, 'percentChange7d': 0, 'lastUpdated': '2025-11-18T07:45:00.000Z', 'percentChange30d': 0, 'percentChange60d': 0, 'percentChange90d': 0, 'fullyDilluttedMarketCap': 1898062762931.95, 'marketCapByTotalSupply': 19950356.0, 'dominance': 58.2121, 'turnover': 0.06296427, 'ytdPriceChangePercentage': -4.2743, 'percentChange1y': -1.58194594}, {'name': 'ETH', 'price': 29.89877549422844, 'volume24h': 37557632.24951095, 'volume7d': 262902967.74134308, 'volumePercentChange': 52.3325, 'volume30d': 903898801.4944359, 'marketCap': 596491215.0739334, 'selfReportedMarketCap': 0.0, 'percentChange1h': -0.54832, 'percentChange24h': -0.035865, 'percentChange7d': 1.182863, 'lastUpdated': '2025-11-18T07:45:00.000Z', 'percentChange30d': 8.713412, 'percentChang

In [16]:
missing_coins = []
for coin in c_data:
    symbol = coin["symbol"].upper()
    if symbol not in s_symbols:
        missing_coins.append({
            "name": coin["name"],
            "symbol": symbol,
            "volume24h": coin["quotes"][2]["volume24h"]
        })

Сортируем и смотрим на список

In [20]:
missing_coins_sorted = sorted(missing_coins, key=lambda x: x["volume24h"], reverse=True)

In [22]:
for i in missing_coins_sorted[0:3]:
    print(i)

{'name': 'First Digital USD', 'symbol': 'FDUSD', 'volume24h': 7804115692.206893}
{'name': 'BNB', 'symbol': 'BNB', 'volume24h': 4450110075.378638}
{'name': 'Kite', 'symbol': 'KITE', 'volume24h': 1044600879.6204245}


Теперь переместим список монет в базу данных, где выполнял первое задание

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname="evercode_lab", user="postgres", password="Aa110604", host="localhost", port="5432")
cursor = conn.cursor()

cursor.execute("""
create table if not exists missing_coin (
    id serial primary key,
    name text,
    symbol text,
    volume24h numeric
)
""")
conn.commit()
# Очистим таблицу, чтоб не дублировать данные
cursor.execute("truncate table missing_coin restart identity") # restart identity - сбрасывает связанные sequence
conn.commit()

# Вставляем сами данные
for coin in missing_coins_sorted:
    cursor.execute(
        "insert into missing_coin (name, symbol, volume24h) values(%s, %s, %s)", # с плейсхолдерами
        (coin["name"], coin["symbol"], coin["volume24h"])
    )

conn.commit()

Проверим данные, выполнив запрос к бд и закроем соединение

Всего нашлось монет

In [31]:
cursor.execute("select count(*) from missing_coin")
cnt = cursor.fetchone()[0]
print(cnt)

532


In [32]:
cursor.execute("select * from missing_coin limit 10")
rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()
conn.close()

(1, 'First Digital USD', 'FDUSD', Decimal('7804115692.206893'))
(2, 'BNB', 'BNB', Decimal('4450110075.378638'))
(3, 'Kite', 'KITE', Decimal('1044600879.6204245'))
(4, '48 Club Token', 'KOGE', Decimal('622205660.6552982'))
(5, 'Marina Protocol', 'BAY', Decimal('341599925.5829847'))
(6, 'Horizen', 'ZEN', Decimal('338071506.77540565'))
(7, 'Quq', 'QUQ', Decimal('290282920.24014294'))
(8, 'APRO', 'AT', Decimal('267442123.20176795'))
(9, 'MetaArena', 'TIMI', Decimal('256706880.81863898'))
(10, 'Momentum', 'MMT', Decimal('219783201.77488273'))
